In [92]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [93]:
import numpy as np
import pandas as pd

In [94]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [95]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [96]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [97]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [98]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [99]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [100]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [101]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [102]:
#see column names and types of measurement
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c['type'])


id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [103]:
#see column names and types of station
columns = inspector.get_columns('station')
for c in columns: 
    print(c['name'], c['type'])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [153]:
x = session.query(measurement)
x_df = pd.read_sql(x.statement, x.session.bind)
x_df

,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0
...,...,...,...,...,...
19545,19546,USC00516128,2017-08-19,0.09,71.0
19546,19547,USC00516128,2017-08-20,NaN,78.0
19547,19548,USC00516128,2017-08-21,0.56,76.0
19548,19549,USC00516128,2017-08-22,0.50,76.0


In [158]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

twelve_months_ago = dt.date.today() - dt.timedelta(days=365)
last_12_months = session.query(measurement.date, measurement.station, measurement.tobs).filter(measurement.date.between('2016-11-15', '2017-11-15'))
    
last_12_months_df = pd.read_sql(last_12_months.statement, last_12_months.session.bind)
last_12_months_df






,date,station,tobs
0,2016-11-15,USC00519397,77.0
1,2016-11-16,USC00519397,77.0
2,2016-11-17,USC00519397,77.0
3,2016-11-18,USC00519397,77.0
4,2016-11-19,USC00519397,75.0
...,...,...,...
1690,2017-08-19,USC00516128,71.0
1691,2017-08-20,USC00516128,78.0
1692,2017-08-21,USC00516128,76.0
1693,2017-08-22,USC00516128,76.0


In [160]:
# Calculate the date 1 year ago from the last data point in the database
last_date = session.query(measurement.date).order_by(measurement.date.desc()).first()
# year_past = dt.date(last_date) - dt.timedelta(days=365)
last_date

('2017-08-23')

In [170]:
# Perform a query to retrieve the data and precipitation scores
precipitation = session.query(measurement.date, measurement.prcp)
# Save the query results as a Pandas DataFrame and set the index to the date column
preciptitation_df = pd.read_sql(precipitation.statement, precipitation.session.bind).set_index('date')
# Sort the dataframe by date
preciptitation_sorted = preciptitation_df.sort_values('date')
# Use Pandas Plotting with Matplotlib to plot the data


,prcp
date,
2010-01-01,0.08
2010-01-01,0.21
2010-01-01,0.15
2010-01-01,0.15
2010-01-01,0.05
...,...
2017-08-22,0.50
2017-08-23,0.00
2017-08-23,0.00


In [173]:
# Use Pandas to calcualte the summary statistics for the precipitation data
preciptitation_sorted.describe()

,prcp
count,18103.000000
mean,0.160644
std,0.468746
min,0.000000
25%,0.000000
50%,0.010000
75%,0.110000
max,11.530000


In [108]:
# Design a query to show how many stations are available in this dataset?
stations = session.query(measurement).group_by(measurement.station).count()
stations


9

In [138]:
# What are the most active stations? (i.e. what stations have the most rows)?


results = session.query(func.count(measurement.station), measurement.station).\
    group_by(measurement.station).\
    order_by(func.count(measurement.station).desc()).all()
        
# # # List the stations and the counts in descending order.
results


[(2772, 'USC00519281'),
 (2724, 'USC00519397'),
 (2709, 'USC00513117'),
 (2669, 'USC00519523'),
 (2612, 'USC00516128'),
 (2202, 'USC00514830'),
 (1979, 'USC00511918'),
 (1372, 'USC00517948'),
 (511, 'USC00518838')]

In [175]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
sel = [measurement.station,
      func.min(measurement.tobs),
      func.max(measurement.tobs),
      func.avg(measurement.tobs)]

active_station_query = session.query(*sel).\
    filter(measurement.station == 'USC00519281')

active_station_df = pd.read_sql(active_station_query.statement, active_station_query.session.bind)
active_station_df

,station,min_1,max_1,avg_1
0,USC00519281,54.0,85.0,71.663781


In [188]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
dates = session.query(func.min(measurement.date), func.max(measurement.date)).all()

high_station = session.query(measurement.date, measurement.tobs).\
    filter(measurement.date.between('2016-08-23', '2017-08-23'))
           
high_station_df = pd.read_sql(high_station.statement, high_station.session.bind)
high_station_df


,date,tobs
0,2016-08-23,81.0
1,2016-08-24,79.0
2,2016-08-25,80.0
3,2016-08-26,79.0
4,2016-08-27,77.0
...,...,...
2225,2017-08-19,71.0
2226,2017-08-20,78.0
2227,2017-08-21,76.0
2228,2017-08-22,76.0


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
